In [0]:
!pip install transformers

     |████████████████████████████████| 184kB 4.9MB/s 
     |████████████████████████████████| 1.0MB 40.4MB/s 
     |████████████████████████████████| 870kB 50.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=e023338d58a09b37160f8f6fe31dff687bba7707e9896e28fd784a61837dd004
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# 加载预训练模型 tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 标记化输出
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', 'henson', 'was', 'a', 'puppet', '##eer', '[SEP]']


In [0]:

# 掩码一个标记，我们将尝试用' BertForMaskedLM '预测回来
masked_index = 8
tokenized_text[masked_index] = '[MASK]'
assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer', '[SEP]']

# 将标记转换为词汇表索引
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
print(indexed_tokens)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]



[101, 2040, 2001, 3958, 27227, 1029, 102, 3958, 103, 2001, 1037, 13997, 11510, 102]


In [0]:
# 将输入转换为PyTorch张量
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [0]:
# 加载预训练模型 (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

# ：如果你有GPU，把所有东西都放在cuda上
tokens_tensor = tokens_tensor.to('cuda')
segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

# 预测所有的tokens
with torch.no_grad():
    outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    predictions = outputs[0]
    print(outputs)
    print(predictions)

(tensor([[[ -7.8798,  -7.7874,  -7.7861,  ...,  -7.0438,  -6.7454,  -4.6013],
         [-13.3633, -13.7694, -13.7819,  ..., -11.8128, -11.1635, -13.8906],
         [-10.9775, -10.5383, -10.9659,  ..., -11.5549,  -8.0309,  -6.3979],
         ...,
         [ -5.2284,  -5.6572,  -5.3550,  ...,  -3.4507,  -3.8718,  -8.6904],
         [ -8.5290,  -8.4146,  -9.0744,  ...,  -7.1710,  -6.9877,  -6.1301],
         [-12.5968, -12.3769, -12.4222,  ..., -10.1020,  -9.8764,  -9.4495]]],
       device='cuda:0'),)
tensor([[[ -7.8798,  -7.7874,  -7.7861,  ...,  -7.0438,  -6.7454,  -4.6013],
         [-13.3633, -13.7694, -13.7819,  ..., -11.8128, -11.1635, -13.8906],
         [-10.9775, -10.5383, -10.9659,  ..., -11.5549,  -8.0309,  -6.3979],
         ...,
         [ -5.2284,  -5.6572,  -5.3550,  ...,  -3.4507,  -3.8718,  -8.6904],
         [ -8.5290,  -8.4146,  -9.0744,  ...,  -7.1710,  -6.9877,  -6.1301],
         [-12.5968, -12.3769, -12.4222,  ..., -10.1020,  -9.8764,  -9.4495]]],
       device='cu

In [0]:

# 证实我们能够预测到'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
assert predicted_token == 'henson'
print('Predicted token is:',predicted_token)

Predicted token is: henson
